<a href="https://colab.research.google.com/github/asanaullah2015/cot5600/blob/master/hw_1/problem_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## HW 1 ##

### Problem 2 ###

**Some definitions**

Let 

$$M\in\mathbb{R}^{n\times n}$$ be an arbitrary matrix.  

Let $$p(x)=a_0 + a_1 x + a_2 x^2 + \ldots + a_n x^n \in\mathbb{R}[x]$$ be an arbitrary polynomial of degree less or equal to $n$.

The above polynomial can be used to define a matrix function that takes matrices as input and outputs matrices as follows: 

$$p(M) = a_0 I + a_1 M + \ldots + a_n M^n,$$ 

that is, each monomial $x^k$ is substituted by the corresponding matrix power $M^k$.

We say that a polynomial $p(x)$ annihilates a matrix $M\in\mathbb{R}^{n\times n}$ iff $p(M)=\boldsymbol{0}$, where $\boldsymbol{0}$ is the zero matrix.

**Task**

The task is to write a function ```annihilate_poly``` that takes as input an arbitrary square numpy array $M$ and outputs a vector whose cofficients are the coefficients of a (non-trivial) polynomial that annihilates $M$.  One-trivial means that its is not the zero polynomial which maps every matrix to the zero matrix.

**Hint**

You can reduce the problem to finding a linear dependance relationship between the $n+1$ vectors 

$$\mathrm{vec}(I), \mathrm{vec}(M), \mathrm{vec}(M^2),\ldots,\mathrm{vec}(M^n)\in\mathrm{R}^{n^2}.$$



The operation $\mathrm{vec}$ turns a square matrix $M\in\mathbb{R}^{n\times n}$ into a vector $v\in\mathbb{R}^{n^2}$ by first listing the entries of the first row, then those of the second row etc.

Update: 

To solve this problem, you have to compute the null space of the matrix $A\in \mathbb{R}^{n^2\times (n+1)}$ whose columns are the vectors $\mathrm{vec}(M^k)$ for $k\in\{0,\ldots,n\}$.


(This is not needed: 

If you don't remeber how to compute the find a linear dependance relationship, check out this stackoverflow post: https://math.stackexchange.com/questions/2198960/finding-linear-dependence-relation

You can use https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.solve.html to solve the resulting matrix equation.)

**Task**

Write a function ```annihilate_min_deg_poly``` that computes a non-trivial polynomial that annihilates a given square matrix and has the smallest possible degree.  Recall that a polynomial $p(x)$ has degree $d$ if the coefficient $a_{d+1}=\ldots=a_n=0$.

---
Below is the cleaned up code

---

In [103]:
import numpy as np
from scipy.linalg import null_space

def annihilate_poly(M, n, k, powersOfM):
  #M, matrix, of nxn dimension, k degree of desired polynomial that annihilates
  #returns -1 if none exists, else returns coefficients in a vector
  
  vecPowers = np.empty((n*n, k+1))
  for i in range (0, k+1):
    vecPowers[:,i] = np.reshape(powersOfM[i], (n*n))
  
  ns = null_space(vecPowers)
  if (ns.size == 0):
    return -1
  return ns

def annihilate_min_deg_poly (M, n):
  #finds minimum degree annihilating polynomial of M, an nxn matrix
  powersOfM = np.empty((n+1, n, n))
  powersOfM[0] = np.identity(n);
  for i in range(1,n+1):
    powersOfM[i] = np.matmul(powersOfM[i-1], M)
  
  coeff = sillyFunction(M, n, 0, n, powersOfM)
  print("The minimum degree annihilating polynomial is of degree ")
  print(np.prod(coeff.shape) - 1)
  t = "p(M) = "
  for i in range (0, np.prod(coeff.shape) - 1):
    t+= str(coeff[i]) + "*M^" + str(i) + " + "
  t+= str(coeff[np.prod(coeff.shape) - 1]) \
    + "*M^" + str(np.prod(coeff.shape) - 1)
  
  print(t)
  print("Evaluation of p(M)")
  vecPowers = np.empty((n*n, coeff.size))
  for i in range (0, coeff.size):
    vecPowers[:,i] = np.reshape(powersOfM[i], (n*n))
  np.reshape(coeff, (coeff.size, 1))

  print("p(M) = \n")
  print(np.reshape(np.matmul(vecPowers, coeff), (n,n)))

def sillyFunction(M, n, i, j, powersOfM):
  #binary search for degree of min annihilating polynomial
  if i == j:
    ns = annihilate_poly(M, n, i, powersOfM)
    return ns[:, 0]  
  k = (i+j)//2
  v = annihilate_poly(M, n, k, powersOfM)
  if np.isscalar(v) and v == -1:
    return sillyFunction(M, n, k+1, j, powersOfM)
  return sillyFunction(M, n, i, k, powersOfM)

M = [[5,9,9,7],
     [5,5,4,5],
     [6,5,11,1],
     [1,5,3,7]]
n = 4
"""
#this is for generating random n and M, 
#for high n, rounding error becomes high
#for random matrices, the min degree polynomial is usually degree n
n = np.random.randint(1,10)
M = np.random.randint(1,10,size=(n,n))
print(n)
print(M)
"""
annihilate_min_deg_poly(M, n);

The minimum degree annihilating polynomial is of degree 
4
p(M) = -0.8819179317604852*M^0 + 0.33071922441238255*M^1 + 0.32815550949475525*M^2 + -0.07178401770197779*M^3 + 0.002563714917927784*M^4
Evaluation of p(M)
p(M) = 

[[ 5.48538992e-12 -2.30215846e-12 -2.21689334e-12 -1.79056769e-12]
 [-1.29318778e-12  5.45696821e-12 -8.81072992e-13 -1.29318778e-12]
 [-1.50635060e-12 -1.13686838e-12  3.86535248e-12 -2.84217094e-14]
 [-1.56319402e-13 -1.32160949e-12 -6.53699317e-13  4.73221462e-12]]


---
---

**The below code has comments to show how I got the final code, which is in the above code snippet**



In [41]:
#@title Default title text
def annihilate_poly(M, n, k, powersOfM):
  #M, matrix, of nxn dimension, k degree of desired polynomial that annihilates
  #returns -1 if none exists, else returns coefficients in a vector
  import numpy as np

  #M = [[5,9,9,7],
  #    [5,5,4,5],
  #    [6,5,2,1],
  #    [1,5,3,7]]
  #n = 4
  #powersOfM = np.empty((k+1, n, n))
  #powersOfM[0] = np.identity(n);
  #for i in range(1,k+1):
  #  powersOfM[i] = np.matmul(powersOfM[i-1], M)
    #print(powersOfM[i-1])
  #print(powersOfM[n])

  vecPowers = np.empty((n*n, k+1))
  for i in range (0, k+1):
    vecPowers[:,i] = np.reshape(powersOfM[i], (n*n))
  #print(vecPowers)

  from scipy.linalg import null_space
  ns = null_space(vecPowers)
  #print(ns)
  #print(np.matmul(vecPowers, ns[:, 0]))
  if (ns.size == 0):
    return -1
  return ns

def annihilate_min_deg_poly (M, n):
  #finds minimum degree annihilating polynomial of M, an nxn matrix
  powersOfM = np.empty((n+1, n, n))
  powersOfM[0] = np.identity(n);
  for i in range(1,n+1):
    powersOfM[i] = np.matmul(powersOfM[i-1], M)
  
  
  
  coeff = sillyFunction(M, n, 0, n, powersOfM)
  print("The minimum degree annihilating polynomial is of degree ")
  print(np.prod(coeff.shape) - 1)
  t = "p(M) = "
  for i in range (0, np.prod(coeff.shape) - 1):
    t+= str(coeff[i]) + "*M^" + str(i) + " + "
  t+= str(coeff[np.prod(coeff.shape) - 1]) \
    + "*M^" + str(np.prod(coeff.shape) - 1)
  
  print(t)
  print("Evaluation of p(M)")
  vecPowers = np.empty((n*n, coeff.size))
  for i in range (0, coeff.size):
    vecPowers[:,i] = np.reshape(powersOfM[i], (n*n))
  np.reshape(coeff, (1, coeff.size))

  print("p(M) = \n" + str(np.matmul(vecPowers, coeff)))

def sillyFunction(M, n, i, j, powersOfM):
  #binary search for degree of min annihilating polynomial
  if i == j:
    ns = annihilate_poly(M, n, i, powersOfM)
    return ns[:, 0]  
  k = (i+j)//2
  v = annihilate_poly(M, n, k, powersOfM)
  if v == -1:
    return sillyFunction(M, n, k+1, j, powersOfM)
  return sillyFunction(M, n, i, k, powersOfM)

M = [[5,9,9,7],
     [5,5,4,5],
     [6,5,2,1],
     [1,5,3,7]]
n = 4
annihilate_min_deg_poly(M, n);

The minimum degree annihilating polynomial is of degree 
4
p(M) = 0.7891014296192499*M^0 + 0.6107141383462158*M^1 + -0.05246685037339675*M^2 + -0.039874806283793154*M^3 + 0.0020986740149364635*M^4
Evaluation of p(M)
p(M) = 
[ 2.79953838e-12 -1.27897692e-12 -1.74793513e-12 -9.94759830e-13
 -8.52651283e-13  2.60058641e-12 -4.54747351e-13 -6.67910172e-13
 -1.24344979e-12 -7.24753590e-13  3.26849658e-12  2.84217094e-13
  1.42108547e-13 -7.81597009e-13 -4.68958206e-13  1.81898940e-12]
